# A Demonstration of DiscoCH
## Switzerland's First Operationally Ready National Early Leaf Discoloration Detection Tool

*Trees that experienced early discoloration during the drought year 2018 presented significant physiological stress (Wohlgemuth et al., 2020; Walthert et al., 2021; Frei et al.,
64 2022), reduced growth (Klesse et al., 2024; Rohner et al., 2021) and increased rates of dieback and mortality (Obladen et al., 2021; Neycken et al., 2024) in the following years*

DiscoCH, developed by the WSL Forest Resources and Managment GIS Group as a part of the FOEN/NFI Funded Project *Drought Assesment in Swiss Forests*, is a remote sensing based early leaf discoloration detection model trained and tested on aproximately 2000 field observations of early leaf discoloration made in Switzerland's European Beech forests between 2017 and 2023.
The model is trained to accurately predict whether trees within a 10 m pixel are seasonaly discolored regardless of disturbance or crown damage from previous years.

***DiscoCH fundamentally simulates a regular field observation of discoloration across all of Switzerland's dominant European Beech forests.***

Additional information on the basis for the DiscoCH model and its historical application can be found at this link: [Bloom et al., in Review](http://dx.doi.org/10.2139/ssrn.5343751)

**This notebook demonstrates an application of DiscoCH to a small area in the Canton of Schaffhausen. We use the historical SwissEO Sentinel-2 Time Series to simulate how discoloration developed over the course of the summer 2018 as if new data was becoming available in near real-time.**

## 1. Notebook Setup
This cell imports DiscoCH tools and other necessary libraries. It assumes that src/ is on your Python path.

In [1]:
from disco_ch.stac_pull import (
new_image_check,
load_minmax_rasters,
update_vi_min_max,
pull_closest_from_stac,
normalize_vis,
build_template
)
from disco_ch.apply_model_stac import apply_disco
import disco_ch.user_defined_params as udp
import os
from pathlib import Path

## 2. Check and prepare annual Min-Max Vegetation Indices
This cell checks for min-max data 

In [ ]:
print("Checking for Min-Max data")

vi_min, vi_max = None, None

items_to_process, existing_meta, processed_dates = new_image_check(
    udp.date_of_interest,
    udp.existing_data,
    udp.stac_location,
)

if items_to_process is None:
    print("Loading existing Min-Max rasters")
    vi_min, vi_max = load_minmax_rasters(
        udp.year_of_interest, 
        udp.existing_data
    )
else:
    print(f"Processing {len(items_to_process)} new STAC items")

    template = build_template(ch_template, udp.bounding_box)
    
    update_vi_min_max(
        items_to_process,
        udp.year_of_interest,
        udp.existing_data,
        udp.forest_mask,
        template,
    )
    vi_min, vi_max = load_minmax_rasters(
        udp.year_of_interest, 
        udp.existing_data)